In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import src.config as config

import hopsworks

project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

feature_store = project.get_feature_store()

feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/423065
Connected. Call `.close()` to terminate connection gracefully.


In [7]:
try:
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')

feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view already existed. Skip creation.


In [8]:
ts_data, _ = feature_view.training_data(
    description='Times-series hourly taxi rides',
)

ts_data.sort_values(by=["pickup_location_id","pickup_hour"], inplace=True)
ts_data

: 

In [ ]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28,
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

100%|██████████| 262/262 [00:55<00:00,  4.73it/s]


features_and_target.shape=(91579, 675)


In [ ]:
print(f'{features_and_target.shape=}')

features_and_target.shape=(91579, 675)


In [ ]:
from datetime import timedelta, datetime
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# training data -> from January 2022 up until 2 months ago
# test data -> last 2 months

cutoff_date = timezone('UTC').localize((datetime.today() - timedelta(days=28*1)))
# cutoff_date = pd.to_datetime(datetime.now() - timedelta(days=28*1))

print(f'{cutoff_date=}')

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=datetime.datetime(2023, 12, 30, 12, 27, 35, 600852, tzinfo=<UTC>)
X_train.shape=(84097, 674)
y_train.shape=(84097,)
X_test.shape=(7482, 674)
y_test.shape=(7482,)


In [ ]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:

    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves",2,256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),
    }

    tss = TimeSeriesSplit(n_splits=4)
    scores = []
    for train_index, val_index in tss.split(X_train):
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]

        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)

        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)

    return np.array(scores).mean()

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1)

[I 2024-01-27 12:28:56,731] A new study created in memory with name: no-name-454ca7c3-4d97-497d-bbc8-c288d48810b8
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [ ]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 116, 'feature_fraction': 0.6724402029484466, 'bagging_fraction': 0.5959370704450638, 'min_child_samples': 100}


In [ ]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.6724402029484466, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6724402029484466
[LightGBM] [Warning] bagging_fraction is set=0.5959370704450638, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5959370704450638
[LightGBM] [Warning] feature_fraction is set=0.6724402029484466, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6724402029484466
[LightGBM] [Warning] bagging_fraction is set=0.5959370704450638, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5959370704450638
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.255985 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 171900
[LightGBM] [Info] Number of data points in the train set: 84097, number of used features: 676
[LightGBM] [Info] Start training from scor

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x0000016EEF3F1430>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.5959370704450638,
                               feature_fraction=0.6724402029484466,
                               min_child_samples=100, num_leaves=116))])

In [ ]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.6724402029484466, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6724402029484466
[LightGBM] [Warning] bagging_fraction is set=0.5959370704450638, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5959370704450638
test_mae=4.3414


In [ ]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')



['C:\\Users\\micha\\taxi_demand_predictor_ml2\\models\\model.pkl']

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [ ]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name = "taxi_demand_predictor_next_hour",
    metrics = {"test_mae": test_mae},
    description = "LightGBM regressor with a bit of hyper-parameter tuning",
    input_example = X_train.sample(),
    model_schema=model_schema
)

model.save(MODELS_DIR / 'model.pkl')

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/423065/models/taxi_demand_predictor_next_hour/1


Model(name: 'taxi_demand_predictor_next_hour', version: 1)

Schema(type: 'columnar')